In [1]:
import random
import numpy as np
from queryParser import queryParser,listToQuery
import psycopg2
from moz_sql_parser import parse
from moz_sql_parser import format
from functions import *

# Enter here the percent of top-grated individuals to be retained for the next generation (range 0-1)
GRADED_RETAIN_PERCENT = 0.5
 
# Enter here the chance for a non top-grated individual to be retained for the next generation (range 0-1)
CHANCE_RETAIN_NONGRATED = 0.1
 
# Number of individual in the population
POPULATION_COUNT = 50
 
# Maximum number of generation before stopping the script
GENERATION_COUNT_MAX = 20
 
#Number of top-grated individuals to be retained for the next generation
GRADED_INDIVIDUAL_RETAIN_COUNT = int(POPULATION_COUNT * GRADED_RETAIN_PERCENT)



def get_random_population(Relations):
    """ Create a new random population, made of `POPULATION_COUNT` individual. """
    return [get_random_state(Relations) for _ in range(POPULATION_COUNT)]

 
"""""def average_population_cost(population,parsed_query):
   Return the average fitness of all individual in the population
    total = 0
    for individual in population:
        total += get_cost(individual,mode,parsed_query)
    return total / POPULATION_COUNT"""
  
def grade_population(population,cursor):
    """ Grade the population. Return a list of tuple (individual, fitness) sorted from most graded to less graded. """
    graded_individual = []
    for individual in population:
        graded_individual.append((individual, get_cost(listToQuery(individual,get_indice(old_state),parsed_query),cursor)))
    return sorted(graded_individual, key=lambda x: x[1], reverse=False)



def get_local_min(sorted_population):
    min=100000000
    for state,cost in sorted_population:
        if cost<min :
            min=cost
            
    return min    



def evolve_population(sorted_population):
    """ Make the given population evolving to his next generation. """
  
    # Get individuals sorted by grade (top first), the average cost 
    #average_cost = 0
    graded_population = []
    for individual, fitness in sorted_population:
        #average_cost += fitness
        graded_population.append(individual)
    #average_cost /= POPULATION_COUNT
        
    # Filter the top graded individuals
    parents = graded_population[:GRADED_INDIVIDUAL_RETAIN_COUNT]
   
 
    # Randomly add other individuals to promote genetic diversity
    for individual in graded_population[GRADED_INDIVIDUAL_RETAIN_COUNT:]:
        if random.uniform(0, 1) < CHANCE_RETAIN_NONGRATED:
            parents.append(individual)
    
   
    # Crossover parents to create children
    ind_len=len(parents[1])
    parents_len = len(parents)
    desired_len = POPULATION_COUNT - parents_len
    children = []
    while len(children) < desired_len:
        parent=random.sample(parents, 1)[0]
        child=move(parent)
        children.append(child)
    
    # The next generation is ready
    parents.extend(children)

    return parents

def Genetic_algorithm(Relations,cursor):

    #generate the initial population
    population = get_random_population(Relations)
    #print('Initial population',population)
    
    i = 0
    #costs_avg = []
    #locals_min=[]
    #Average cost of the initial population
    #average_cost = average_population_cost(population,mode,parsed_query)
    #costs_avg.append(average_cost)

    # Make the population evolve
    while  i < GENERATION_COUNT_MAX:
        # grade population
        sorted_population=grade_population(population,cursor)
        #local_min_state=get_local_min(sorted_population)
        #locals_min.append(local_min_state)
        population = evolve_population(sorted_population)
        #costs_avg.append(average_cost)
        i += 1

    min_state=grade_population(population,cursor)[0]
            
    return min_state

 

   


In [2]:
#Execution


input="SELECT d.d_year,s.s_city,p.p_brand1,SUM(l.lo_revenue - l.lo_supplycost) AS profit FROM date d,customer c,supplier s,part p ,lineorder l WHERE l.lo_custkey = c.c_custkey AND l.lo_suppkey = s.s_suppkey  AND l.lo_partkey = p.p_partkey AND l.lo_orderdate = d.d_datekey AND c.c_region = 'AMERICA' AND s.s_nation = 'UNITED STATES' AND (d.d_year = 1997 OR d.d_year = 1998)  AND p.p_category = 'MFGR#14' GROUP BY  d.d_year, s.s_city, p.p_brand1 ORDER BY d.d_year,s.s_city,p.p_brand1;"
R,mode,parsed_query=queryParser(input)
R=delete_fact(R)
#print('R',R)
#Apply algorithm
min_state,costs_avg,locals_min=Genetic_algorithm(R,mode,parsed_query)
solution=listToQuery(list(min_state)[0],get_indice(list(min_state)[0]),mode,parsed_query)
#Print results
print("min state" ,min_state)
print('Solution:',solution, 'with cost ', min_state[1])
#print('locals minimum :',locals_min )
print('Execution time in milisecond:',(time.time() - start_time)*1000)
print('Postgres Cost: ',postgres_cost(input))
#trace the graph localMinCost=f(generation)        
import pygal
line_chart = pygal.Line(show_dots=False, show_legend=False)
line_chart.title = 'cost evolution'
line_chart.x_title = 'Generations'
line_chart.y_title = 'local_min_cost'
line_chart.add('Fitness', locals_min)
line_chart.render_to_file('GA.svg')
#trace the graph averageCost=f(generation)   
line_chart2 = pygal.Line(show_dots=False, show_legend=False)
line_chart2.title = 'cost evolution'
line_chart2.x_title = 'Generations'
line_chart2.y_title = 'average_cost'
line_chart2.add('Fitness', costs_avg)
line_chart2.render_to_file('GA2.svg')

            

taaaable aveeeec aliaaas
saaaaans joiiiiin
min state (['lineorder', 'part', 'supplier', 'date', 'customer'], 3442532.71)
Solution: SELECT d.d_year, s.s_city, p.p_brand1, SUM(l.lo_revenue - l.lo_supplycost) AS profit FROM lineorder AS l JOIN part AS p ON l.lo_partkey = p.p_partkey JOIN supplier AS s ON l.lo_suppkey = s.s_suppkey JOIN date AS d ON l.lo_orderdate = d.d_datekey JOIN customer AS c ON l.lo_custkey = c.c_custkey WHERE l.lo_custkey = c.c_custkey AND l.lo_suppkey = s.s_suppkey AND l.lo_partkey = p.p_partkey AND l.lo_orderdate = d.d_datekey AND c.c_region = 'AMERICA' AND s.s_nation = 'UNITED STATES' AND (d.d_year = 1997 OR d.d_year = 1998) AND p.p_category = 'MFGR#14' GROUP BY d.d_year, s.s_city, p.p_brand1 ORDER BY d.d_year,s.s_city,p.p_brand1 with cost  3442532.71
Execution time in milisecond: 91128.0426979065
Postgres Cost:  3442532.71


In [1]:
from moz_sql_parser import parse
from moz_sql_parser import format
input="select supp_nation, cust_nation,l_year, sum(volume) as revenue from (select n1.n_name as supp_nation, n2.n_name as cust_nation, extract(year from l_shipdate) as l_year,l_extendedprice * (1 - l_discount) as volume from supplier,lineitem,orders,customer,nation n1,nation n2 where s_suppkey = l_suppkey and o_orderkey = l_orderkey and c_custkey = o_custkey and s_nationkey = n1.n_nationkey and c_nationkey = n2.n_nationkey and ((n1.n_name = ':1' and n2.n_name = ':2') or (n1.n_name = ':2' and n2.n_name = ':1')) and l_shipdate between date '1995-01-01' and date '1996-12-31') as shipping group by supp_nation, cust_nation, l_year order by supp_nation, cust_nation, l_year;"
parsed_query=parse(input)
print(parsed_query)

{'select': [{'value': 'supp_nation'}, {'value': 'cust_nation'}, {'value': 'l_year'}, {'value': {'sum': 'volume'}, 'name': 'revenue'}], 'from': {'value': {'select': [{'value': 'n1.n_name', 'name': 'supp_nation'}, {'value': 'n2.n_name', 'name': 'cust_nation'}, {'value': {'extract': ['year', 'l_shipdate']}, 'name': 'l_year'}, {'value': {'mul': ['l_extendedprice', {'sub': [1, 'l_discount']}]}, 'name': 'volume'}], 'from': ['supplier', 'lineitem', 'orders', 'customer', {'value': 'nation', 'name': 'n1'}, {'value': 'nation', 'name': 'n2'}], 'where': {'and': [{'eq': ['s_suppkey', 'l_suppkey']}, {'eq': ['o_orderkey', 'l_orderkey']}, {'eq': ['c_custkey', 'o_custkey']}, {'eq': ['s_nationkey', 'n1.n_nationkey']}, {'eq': ['c_nationkey', 'n2.n_nationkey']}, {'or': [{'and': [{'eq': ['n1.n_name', {'literal': ':1'}]}, {'eq': ['n2.n_name', {'literal': ':2'}]}]}, {'and': [{'eq': ['n1.n_name', {'literal': ':2'}]}, {'eq': ['n2.n_name', {'literal': ':1'}]}]}]}, {'between': ['l_shipdate', {'date': {'literal':

In [3]:
import random
import numpy as np
d2=[[7,6,5,1],[7,6,4,2],[7,6,4],[3,2]]
r=[6,5,1,4]
done=False
result=[]
for e in d2 :
        i=0
        while i<(len(e)-1):
            if(e[i] in r):
                print ("yes")
                result.append(e[i:])
                r.remove(e[i])
                if (e[i+1] in r):
                    print('hhhhh')
                    result[i]=[e[i],e[i+1]]
                    #r.remove(e[i+1])
                    
                    
                i+=1    
            else:
                print('nooo')
                i+=1
    
print(result)            
            

nooo
yes
hhhhh


IndexError: list assignment index out of range